In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import re
from copy import deepcopy
import time

In [2]:
BASE_URL = 'https://understat.com/match/'


In [3]:
final_events = []

In [4]:
def format_info(raw_string):
	pattern = r"'(.+?)(?=')"
	pattern_capture = re.findall(pattern, raw_string)[0]
	pattern_capture = pattern_capture.encode('utf-8').decode('unicode_escape')
	return json.loads(pattern_capture)

def process_team_events(timeline_div, team, match_id):
	events = []
	for e in timeline_div.find_all("div", class_="timeline-row"):

		event_type = e.find("i")['title']
		if event_type is None:
			continue
		event_obj = {}
		if event_type == "Substitution":
			players = e.find_all("a", class_="player-name")
			event_obj["p1"] = players[0]['href'].split("/")[-1]
			event_obj["p2"] = players[1]['href'].split("/")[-1]
		else:
			player = e.find("a", class_="player-name")
			event_obj["p1"] = player['href'].split("/")[-1]
		event_obj["type"] = event_type
		event_obj["minute"] = minute
		event_obj["team"] = team
		event_obj["match_id"] = match_id
		events.append(event_obj)
	return events

In [6]:
for match_id in tqdm(range(16677, 18162)):
	time.sleep(0.217)
	url = f"{BASE_URL}{match_id}"
	res = requests.get(url)
	soup = BeautifulSoup(res.content, "lxml")
	try:
		timeline = soup.find("div", class_="timiline-block")
		teams = []
		result_span = soup.find("span", class_="block-match-result")
		for a in result_span.find_all("a"):
			teams.append(a.text)


		match_events = timeline.find_all("div", class_="timiline-container")


		for event in match_events:
			t1_event_raw = event.find("div", class_="timeline-left")
			t2_event_raw = event.find("div", class_="timeline-right")
			minute = event.find("span", class_="minute-value").text[:-1]

			final_events += process_team_events(t1_event_raw, teams[0], match_id)
			final_events += process_team_events(t2_event_raw, teams[1], match_id)
	except Exception as e:
		with open("event_exceptions.log", "a") as f:
			f.write('player_id: {}, exception: {}, request_status: {}\n'.format(
				match_id, str(e), res.status_code))


100%|██████████| 1485/1485 [19:34<00:00,  1.26it/s]


In [71]:
81, 18161

[{'p1': '6', 'type': 'Goal', 'minute': '30', 'team': 'Darmstadt'},
 {'p1': '93', 'type': 'Yellow card', 'minute': '33', 'team': 'Hannover 96'},
 {'p1': '103', 'type': 'Goal', 'minute': '47', 'team': 'Hannover 96'},
 {'p1': '350',
  'p2': '103',
  'type': 'Substitution',
  'minute': '47',
  'team': 'Hannover 96'},
 {'p1': '6', 'type': 'Goal', 'minute': '53', 'team': 'Darmstadt'},
 {'p1': '4', 'type': 'Yellow card', 'minute': '57', 'team': 'Darmstadt'},
 {'p1': '104',
  'type': 'Penalty missed',
  'minute': '58',
  'team': 'Hannover 96'},
 {'p1': '4', 'type': 'Own goal', 'minute': '61', 'team': 'Darmstadt'},
 {'p1': '13',
  'p2': '9',
  'type': 'Substitution',
  'minute': '66',
  'team': 'Darmstadt'},
 {'p1': '14',
  'p2': '10',
  'type': 'Substitution',
  'minute': '73',
  'team': 'Darmstadt'},
 {'p1': '104',
  'p2': '349',
  'type': 'Substitution',
  'minute': '74',
  'team': 'Hannover 96'},
 {'p1': '349', 'type': 'Yellow card', 'minute': '78', 'team': 'Hannover 96'},
 {'p1': '108',
  

In [7]:
df = pd.DataFrame(final_events)
df.to_csv("events/chunk2.csv")

In [9]:
df.shape

(64246, 6)